In [17]:
import random
import numpy as np
import pyautogui
import pandas as pd
import pyperclip
import time
import os
import backtrader as bt
import win32com.client as win32
import pickle
from pandas import Timestamp
from collections import defaultdict
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time
from  自定义函数.MyTT import *
from 自定义函数.likai自定义函数 import  *
from mootdx.reader import Reader
开始时间 = time.time()
np.set_printoptions(threshold=np.inf)  # 打印所有数据
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用于正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用于正常显示负号
pd.set_option('display.unicode.east_asian_width', True)# 设置输出右对齐
pd.set_option('expand_frame_repr', False)# 显示所有的列
pd.set_option('display.max_rows', 5000)# 最多显示数据的行数
pd.set_option('display.float_format', '{:.2f}'.format) # 取消科学计数法,显示完整,可调整小数点后显示位数


In [18]:
def 通达信策略(df) :
    OPEN  =  O  = df.open.values
    HIGH  =  H  = df.high.values
    LOW   =  L  = df.low.values
    CLOSE =  C  = df.close.values
    VOL   =  V  = df.volume.values
    AMO   =       df.amount.values
    TR = to_TR(C,H,L)
    DIF,  DEA,  MACD=  GET_MACD(CLOSE)
    SAR =  TDX_SAR(H,L)
    #### 上面是基础定义 ########
    return df










In [19]:
def 通达信实现(df):
    # 处理每个股票代码的数据，生成新的DataFrame，并将其合并
    result = pd.concat([通达信策略(group) for _, group in df.groupby('股票代码')], ignore_index=True)
    return result


df_base  = pd.read_pickle('00通达信txt数据处理/通达信txt所有股票_不复权.pkl')
print('开始计算指标')
时间戳  = time.time()
df = 通达信实现(df_base)
买入清单dict  = 所有股票交易点导出(  df ,'volume' ,3, 0)
print( '买入清单计算完成,耗时: ' , time.time() -时间戳   )

开始计算指标
买入清单计算完成,耗时:  39.92298102378845


In [20]:
# todo 数据输入 ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

读取股票个数 = None   # 全部导入填 None
初始本金 = 500000
开始日期 = 20240101   # 在一开始倒入的时候能有用，读取的缓存，修改无效
结束日期 = 20241230   # 在一开始倒入的时候能有用，读取的缓存，修改无效



with open("00通达信txt数据处理/交易清单.pkl", "rb") as f:
    买入清单dict = pickle.load(f)

# 买入清单dict = {
#     Timestamp('2024-07-03 00:00:00'): ['000560', '002607', '002600'],
#     Timestamp('2024-07-04 00:00:00'): ['000861', '600811', '000560'],
#     Timestamp('2024-07-05 00:00:00'): ['600518', '600733', '601288']
#     }



df = pd.read_pickle('00通达信txt数据处理/通达信txt所有股票_后复权_时间对齐_回测专用.pkl')
df = df[df['股票代码'].isin(df['股票代码'].unique()[:读取股票个数])]

# todo 定义函数 --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

def feed投喂(df ,cerebro ) :
    股票名字 =  str(df['股票代码2'].iloc[0])
    data = bt.feeds.PandasData(
        dataname=df,
        fromdate=日期(开始日期),
        todate=   日期(结束日期),
        datetime='日期',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=-1  ,
        plot = False    ,    # 超级重要，false就是不用画图了，当我导入了1000只股票，那就非常重要
    )
    cerebro.adddata(data , name= 股票名字 )
    global 计数a
    计数a= 计数a+1
    print(  计数a )


# 中国佣金
class ChinaStockCommission(bt.CommInfoBase):  # 中国股票 佣金与印花税
    params = (
        ('stamp_duty', 0.001),  # 印花税
        ('commission_rate', 0.0015),  # 佣金率  很多干到了 万分之2.5，我设置千分之1 算高了
        ('min_commission', 5),  # 最低佣金
        ('stamp_duty_applicable', True),  # 是否收取印花税
    )

    def _getcommission(self, size, price, pseudoexec):
        # size 为交易股数，price 为交易价格
        commission = abs(size) * price * self.p.commission_rate
        # 佣金不低于最低标准
        commission = max(commission, self.p.min_commission)

        # 如果是卖出交易并且收取印花税，印花税是成交金额的百分比
        if size < 0 and self.p.stamp_duty_applicable:
            stamp_duty = abs(size) * price * self.p.stamp_duty
            commission += stamp_duty

        return commission




In [21]:

# todo 策略定义--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

class 空策略(bt.Strategy):
    def __init__(self):
        '''必选，初始化属性、计算指标等'''
        pass

    def notify_order(self, order):
        '''可选，打印订单信息'''
        pass

    def notify_trade(self, trade):
        '''可选，打印交易信息'''
        pass


    def next(self):
        '''必选，编写交易策略逻辑'''
        当天 = self.datetime.datetime(0)
        print(当天.date())
        print(当天)
        pass

    def stop(self):
        # 策略运行结束时调用
        print('=' * 88)
        print('初始本金:', 初始本金)
        print( '策略结束时： 钱+股票   : ',小数2点(self.broker.getvalue()))
        print( '策略结束时:   钱: ',小数2点(self.broker.getcash()))
        print( '存入的股票个数；' ,  len(self.datas)   )
        # for i in range(len(self.datas)):
        #     print(self.datas[i]._name)
        pass

class 实际策略001beifeng(bt.Strategy):
    def __init__(self):
        '''必选，初始化属性、计算指标等'''
        self.最后一天 = self.datas[0].datetime.date(len(self.datas[0]) - 1)
        global lk_账户
        lk_账户 = defaultdict(dict)
        lk_账户['现金']= 初始本金
        lk_账户['胜_次数'] = 0
        lk_账户['负_次数'] = 0
        lk_账户['总净利润'] = 0

        pass
    def next(self):
        持仓列表 = [abc._name for abc in self.broker.positions if self.broker.getposition(abc).size > 0]
        if self.datetime.date(0) > self.最后一天 -timedelta(days=3) :
            print('=' * 168 + '\n' + '=' * 168 + '\n' + '=' * 168)
            print(self.datetime.date(0))
            print('最后3天,触发全部平仓')
            for i in 持仓列表:
                self.close(i)
            print(持仓列表)
            return



        print('=' * 168 + '\n' + '=' * 168 + '\n' + '=' * 168)
        next当前日期 = self.datetime.datetime(0)
        print(self.datetime.date(0))
        print('钱+股票：', 小数2点(self.broker.getvalue()   ))
        print('钱：',小数2点(self.broker.getcash() ))
        print('我自己计算的钱:' ,  小数2点( lk_账户['现金'] ))

        当天卖出清单 = []
        当天买入清单 = []
        # -------- 开盘前的持仓展示 ---···············
        # -------- 开盘前的持仓展示 ---···············
        # -------- 开盘前的持仓展示 ---···············
        print('---------------------------------')
        print('··· 开盘前持仓展示：')
        持仓总成本 = 0
        持仓总价值 = 0
        for i in 持仓列表:
            该股成本价 = self.getpositionbyname(i).price
            现在价格 = self.getpositionbyname(i).adjbase
            股数 = self.getpositionbyname(i).size
            持股天数 = len(self.getdatabyname(i)) - lk_账户[i]['开仓_bar'] + 1
            该股价值 = 小数2点(现在价格*股数)
            该股成本 = 小数2点(该股成本价*股数)
            该股盈利 = 小数2点(现在价格*股数  - 该股成本价*股数 )

            持仓总成本 = 小数2点(持仓总成本 + 该股成本)
            持仓总价值 = 小数2点(持仓总价值 + 该股价值)
            print(i,'该股成本价:', 小数2点(该股成本价),
                    '现在价格:', 小数2点(现在价格),
                    '股数:', 股数,
                    '持股天数:', 持股天数,
                    '该股价值', 小数2点(该股价值),
                    '该股盈利', 小数2点(该股盈利),
                    )
        print('持仓总成本:' , 小数2点(持仓总成本 ))
        print('持仓总价值:' , 小数2点(持仓总价值 ))




        # -------- 先卖 ---···············
        # -------- 先卖 ---···············
        # -------- 先卖 ---···············
        print('---------------------------------')
        print('··· 先卖：')
        for i in 持仓列表:
            买入时价格 = lk_账户[i]['买入时价格']
            该股成本价 = self.getpositionbyname(i).price
            现在价格 = self.getpositionbyname(i).adjbase
            股数 = self.getpositionbyname(i).size
            持股天数 = len(self.getdatabyname(i)) - lk_账户[i]['开仓_bar'] + 1
            if self.getdatabyname(i).volume[0] ==0 :
                print('当天成交量0',i,'卖出失败')
                continue
            elif  现在价格/该股成本价<0.95  :
                self.close(i)
                当天卖出清单.append(i)
                print('亏5个点，卖出:' ,i  )
            elif  现在价格/该股成本价>1.05 :
                self.close(i)
                当天卖出清单.append(i)
                print('赚5个点，卖出:' ,i  )
            elif  持股天数>=1 :
                self.close(i)
                当天卖出清单.append(i)
                print('持股超过10天，卖出:' ,i  )



        # -------- 后买 ---···············
        # -------- 后买 ---···············
        # -------- 后买 ---···············
        print('---------------------------------')
        print('··· 后买：')
        if next当前日期 in 买入清单dict       :
            # 获取当前日期的买入清单,减去卖出清单,别当天卖,当天买,这个傻逼行为
            当天买入清单 = set(买入清单dict[next当前日期]) - set(当天卖出清单)
        for i in 当天买入清单 :
            if self.getdatabyname(i).volume[0] == 0 :
                print('当天成交量0',i,'买入失败')
                continue
            self.buy(i  ,size=1000 )
            print( '买入',i , self.getdatabyname(i).open[0],self.getdatabyname(i).low[0],self.getdatabyname(i).high[0],self.getdatabyname(i).close[0] , self.getdatabyname(i).volume[0])


        print('---------------------------------')
        print('··· 交易商处理：')
    def notify_order(self, order):

        if order.status in [order.Submitted, order.Accepted]:  # 因为我开启cheat 模式 ,直接购买成功了
            pass
        if order.status in [order.Completed]:   # completed  订单已完成
            print('--- order部分 --- ')
            if order.isbuy():
                lk_账户[order.data._name]['合计size'] =lk_账户[order.data._name].get('合计size', 0) +order.executed.size
                lk_账户['现金']=lk_账户['现金']+(-order.executed.size*order.executed.price-order.executed.comm)
                print('日期:', self.data.num2date(order.executed.dt).date(),
                ' 买入:', order.data._name, '订单', order.ref,
                ' 数量:', order.executed.size,
                ' 买入价格:',小数2点(order.executed.price),
                ' 手续费:', 小数2点(order.executed.comm ),
                ' 现金变动:', 小数2点(-order.executed.size*order.executed.price-order.executed.comm),
                ' 现金余额:',小数2点(lk_账户['现金']),
                )
            elif order.issell():
                lk_账户['现金']=lk_账户['现金']+(-order.executed.size*order.executed.price-order.executed.comm)
                lk_账户[order.data._name]['合计size'] = lk_账户[order.data._name].get('合计size', 0) + order.executed.size
                print('日期:', self.data.num2date(order.executed.dt).date(),
                ' 卖出:', order.data._name, '订单', order.ref,
                ' 数量:', order.executed.size,
                ' 卖出价格', 小数2点(order.executed.price),
                ' 手续费:', 小数2点(order.executed.comm ),
                ' 现金变动', 小数2点(order.executed.size*order.executed.price-order.executed.comm),
                ' 现金余额',小数2点(lk_账户['现金']),
                )

    def notify_trade(self, trade):   # 对于单只股票,首次买入触发,或则完全卖出的时候触发
        if trade.justopened: # 对于单只股票,0仓位买入时候
            print('--- 触发 开仓 ---')
            lk_账户[trade.data._name]['开仓_bar'] = len(self.getdatabyname(trade.data._name))
            lk_账户[trade.data._name]['买入时价格'] =  小数2点(trade.price)
            print('买入: ',
                '日期', self.datetime.date(-1),
                '测试，开仓时间', trade.open_datetime(),
                '订单',trade.ref ,
                '股票',trade.data._name,
                '开仓价格:', 小数2点(trade.price),
                '开仓数量:', trade.size  ,
                '持股天数:', trade.barlen ,
                '毛利润:', 小数2点(trade.pnl) ,
                '净利润:', 小数2点(trade.pnlcomm) ,
                )
            pass
        elif trade.isclosed: # 对于单只股票, 仓位完全变成0 ，平仓
            print('*** 触发 平仓 ***')
            print('平仓: ',
                '日期',self.datetime.date(-1),
                '订单编号',trade.ref ,
                '股票',trade.data._name,
                '卖出股数',lk_账户[trade.data._name]['合计size'],
                '该股成本价:', 小数2点(trade.price),
                '当天卖出价格',小数2点(self.getdatabyname(trade.data._name).close[-1]- 0.01),
                '涨幅',小数2点 ((self.getdatabyname(trade.data._name).close[-1]- 0.01-trade.price)/trade.price *100)  ,
                '持股天数:', trade.barlen  ,
                '毛利润:', 小数2点(trade.pnl),
                '净利润:', 小数2点(trade.pnlcomm),
                )
            if   trade.pnlcomm  > 0   :
                lk_账户['胜_次数'] = lk_账户['胜_次数'] +1
            else  :
                lk_账户['负_次数'] = lk_账户['负_次数'] +1
            lk_账户['总净利润'] = lk_账户['总净利润'] + trade.pnlcomm
            del lk_账户[trade.data._name]

    def stop(self):
        # 策略运行结束时调用
        print('=' * 88)
        print('初始本金:', 初始本金)
        print( '策略结束时： 钱+股票   : ',小数2点(self.broker.getvalue()))
        print( '策略结束时:   钱: ',小数2点(self.broker.getcash()))
        总平仓次数 = lk_账户['胜_次数'] +  lk_账户['负_次数']
        胜率 =  lk_账户['胜_次数'] /  总平仓次数
        print( '总交易次数', 总平仓次数   )
        print('胜率: ',小数2点(胜率) )
        print('总净利润: ',小数2点(lk_账户['总净利润']) )
        print( '获利百分比',  小数2点(  lk_账户['总净利润']/初始本金  )  )
        pass


class 实际策略001(bt.Strategy):
    def __init__(self):
        '''必选，初始化属性、计算指标等'''
        self.最后一天 = self.datas[0].datetime.date(len(self.datas[0]) - 1)
        global lk_账户
        lk_账户 = defaultdict(dict)
        lk_账户['现金']= 初始本金
        lk_账户['胜_次数'] = 0
        lk_账户['负_次数'] = 0
        lk_账户['总净利润'] = 0

        pass
    def next(self):
        持仓列表 = [abc._name for abc in self.broker.positions if self.broker.getposition(abc).size > 0]
        if self.datetime.date(0) > self.最后一天 -timedelta(days=3) :
            print('=' * 168 + '\n' + '=' * 168 + '\n' + '=' * 168)
            print(self.datetime.date(0))
            print('最后3天,触发全部平仓')
            for i in 持仓列表:
                self.close(i)
            print(持仓列表)
            return



        print('=' * 168 + '\n' + '=' * 168 + '\n' + '=' * 168)
        next当前日期 = self.datetime.datetime(0)
        print(self.datetime.date(0))
        print('钱+股票：', 小数2点(self.broker.getvalue()   ))
        print('钱：',小数2点(self.broker.getcash() ))
        print('我自己计算的钱:' ,  小数2点( lk_账户['现金'] ))

        当天卖出清单 = []
        当天买入清单 = []
        # -------- 开盘前的持仓展示 ---···············
        # -------- 开盘前的持仓展示 ---···············
        # -------- 开盘前的持仓展示 ---···············
        print('---------------------------------')
        print('··· 开盘前持仓展示：')
        持仓总成本 = 0
        持仓总价值 = 0
        for i in 持仓列表:
            该股成本价 = self.getpositionbyname(i).price
            现在价格 = self.getpositionbyname(i).adjbase
            股数 = self.getpositionbyname(i).size
            持股天数 = len(self.getdatabyname(i)) - lk_账户[i]['开仓_bar'] + 1
            该股价值 = 小数2点(现在价格*股数)
            该股成本 = 小数2点(该股成本价*股数)
            该股盈利 = 小数2点(现在价格*股数  - 该股成本价*股数 )

            持仓总成本 = 小数2点(持仓总成本 + 该股成本)
            持仓总价值 = 小数2点(持仓总价值 + 该股价值)
            print(i,'该股成本价:', 小数2点(该股成本价),
                    '现在价格:', 小数2点(现在价格),
                    '股数:', 股数,
                    '持股天数:', 持股天数,
                    '该股价值', 小数2点(该股价值),
                    '该股盈利', 小数2点(该股盈利),
                    )
        print('持仓总成本:' , 小数2点(持仓总成本 ))
        print('持仓总价值:' , 小数2点(持仓总价值 ))




        # -------- 先卖 ---···············
        # -------- 先卖 ---···············
        # -------- 先卖 ---···············
        print('---------------------------------')
        print('··· 先卖：')
        for i in 持仓列表:
            买入时价格 = lk_账户[i]['买入时价格']
            该股成本价 = self.getpositionbyname(i).price
            现在价格 = self.getpositionbyname(i).adjbase
            股数 = self.getpositionbyname(i).size
            持股天数 = len(self.getdatabyname(i)) - lk_账户[i]['开仓_bar'] + 1
            if self.getdatabyname(i).volume[0] ==0 :
                print('当天成交量0',i,'卖出失败')
                continue
            elif  现在价格/该股成本价<0.8  :
                self.close(i)
                当天卖出清单.append(i)
                print('亏5个点，卖出:' ,i  )
            elif  现在价格/该股成本价>1.2 :
                self.close(i)
                当天卖出清单.append(i)
                print('赚5个点，卖出:' ,i  )
            elif  持股天数>=1 :
                self.close(i)
                当天卖出清单.append(i)
                print('持股超过1天，卖出:' ,i  )



        # -------- 后买 ---···············
        # -------- 后买 ---···············
        # -------- 后买 ---···············
        print('---------------------------------')
        print('··· 后买：')
        if next当前日期 in 买入清单dict       :
            # 获取当前日期的买入清单,减去卖出清单,别当天卖,当天买,这个傻逼行为
            当天买入清单 = set(买入清单dict[next当前日期]) - set(当天卖出清单)
        for i in 当天买入清单 :
            if self.getdatabyname(i).volume[0] == 0 :
                print('当天成交量0',i,'买入失败')
                continue
            self.buy(i  ,size=1000 )
            print( '买入',i , self.getdatabyname(i).open[0],self.getdatabyname(i).low[0],self.getdatabyname(i).high[0],self.getdatabyname(i).close[0] , self.getdatabyname(i).volume[0])


        print('---------------------------------')
        print('··· 交易商处理：')
    def notify_order(self, order):

        if order.status in [order.Submitted, order.Accepted]:  # 因为我开启cheat 模式 ,直接购买成功了
            pass
        if order.status in [order.Completed]:   # completed  订单已完成
            print('--- order部分 --- ')
            if order.isbuy():
                lk_账户[order.data._name]['合计size'] =lk_账户[order.data._name].get('合计size', 0) +order.executed.size
                lk_账户['现金']=lk_账户['现金']+(-order.executed.size*order.executed.price-order.executed.comm)
                print('日期:', self.data.num2date(order.executed.dt).date(),
                ' 买入:', order.data._name, '订单', order.ref,
                ' 数量:', order.executed.size,
                ' 买入价格:',小数2点(order.executed.price),
                ' 手续费:', 小数2点(order.executed.comm ),
                ' 现金变动:', 小数2点(-order.executed.size*order.executed.price-order.executed.comm),
                ' 现金余额:',小数2点(lk_账户['现金']),
                )
            elif order.issell():
                lk_账户['现金']=lk_账户['现金']+(-order.executed.size*order.executed.price-order.executed.comm)
                lk_账户[order.data._name]['合计size'] = lk_账户[order.data._name].get('合计size', 0) + order.executed.size
                print('日期:', self.data.num2date(order.executed.dt).date(),
                ' 卖出:', order.data._name, '订单', order.ref,
                ' 数量:', order.executed.size,
                ' 卖出价格', 小数2点(order.executed.price),
                ' 手续费:', 小数2点(order.executed.comm ),
                ' 现金变动', 小数2点(order.executed.size*order.executed.price-order.executed.comm),
                ' 现金余额',小数2点(lk_账户['现金']),
                )

    def notify_trade(self, trade):   # 对于单只股票,首次买入触发,或则完全卖出的时候触发
        if trade.justopened: # 对于单只股票,0仓位买入时候
            print('--- 触发 开仓 ---')
            lk_账户[trade.data._name]['开仓_bar'] = len(self.getdatabyname(trade.data._name))
            lk_账户[trade.data._name]['买入时价格'] =  小数2点(trade.price)
            print('买入: ',
                '日期', self.datetime.date(-1),
                '测试，开仓时间', trade.open_datetime(),
                '订单',trade.ref ,
                '股票',trade.data._name,
                '开仓价格:', 小数2点(trade.price),
                '开仓数量:', trade.size  ,
                '持股天数:', trade.barlen ,
                '毛利润:', 小数2点(trade.pnl) ,
                '净利润:', 小数2点(trade.pnlcomm) ,
                )
            pass
        elif trade.isclosed: # 对于单只股票, 仓位完全变成0 ，平仓
            print('*** 触发 平仓 ***')
            print('平仓: ',
                '日期',self.datetime.date(-1),
                '订单编号',trade.ref ,
                '股票',trade.data._name,
                '卖出股数',lk_账户[trade.data._name]['合计size'],
                '该股成本价:', 小数2点(trade.price),
                '当天卖出价格',小数2点(self.getdatabyname(trade.data._name).close[-1]- 0.01),
                '涨幅',小数2点 ((self.getdatabyname(trade.data._name).close[-1]- 0.01-trade.price)/trade.price *100)  ,
                '持股天数:', trade.barlen  ,
                '毛利润:', 小数2点(trade.pnl),
                '净利润:', 小数2点(trade.pnlcomm),
                )
            if   trade.pnlcomm  > 0   :
                lk_账户['胜_次数'] = lk_账户['胜_次数'] +1
            else  :
                lk_账户['负_次数'] = lk_账户['负_次数'] +1
            lk_账户['总净利润'] = lk_账户['总净利润'] + trade.pnlcomm
            del lk_账户[trade.data._name]

    def stop(self):
        # 策略运行结束时调用
        print('=' * 88)
        print('初始本金:', 初始本金)
        print( '策略结束时： 钱+股票   : ',小数2点(self.broker.getvalue()))
        print( '策略结束时:   钱: ',小数2点(self.broker.getcash()))
        总平仓次数 = lk_账户['胜_次数'] +  lk_账户['负_次数']
        胜率 =  lk_账户['胜_次数'] /  总平仓次数
        print( '总交易次数', 总平仓次数   )
        print('胜率: ',小数2点(胜率) )
        print('总净利润: ',小数2点(lk_账户['总净利润']) )
        print( '获利百分比',  小数2点(  lk_账户['总净利润']/初始本金  )  )
        pass




In [22]:
cerebro = bt.Cerebro(
    stdstats=False ,     # 推荐打开，默认是True,观察器选择False,然后我再自己添加，那样就不会有股票数据了
    )
cerebro.broker.set_coc(True) # 用途：适合在当前 bar 内（即在当前时间段的结束时）立即执行基于收盘价的订单的策略

# 调用投喂函数,开始投喂df
if not os.path.exists('回测缓存文件.pkl'):
    print('不存在_回测缓存文件________开始加载')
    计数a = 0
    df['股票代码2'] = df['股票代码']
    df.groupby('股票代码').apply(lambda df: feed投喂(df, cerebro), include_groups=False)
else:
    print('存在_回测缓存文件_______开始欺骗餐')
    df测试 = 欺骗餐()
    data测试 = bt.feeds.PandasData(dataname=df测试)
    cerebro.adddata( data测试 )




存在_回测缓存文件_______开始欺骗餐


In [23]:
cerebro.broker.setcash(初始本金)   # 设置初始资金
cerebro.broker.set_slippage_fixed(fixed=0.01)  # 双边固定0.01滑点 ，不计算到交易成本
cerebro.broker.setcommission(commission=0.002)   # 设置固定双边 手续费比例
# cerebro.broker.addcommissioninfo(ChinaStockCommission())      # 中国股票 佣金与印花税


cerebro.addstrategy(      实际策略001     )     # 添加策略
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')  #交易分析
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')   # 夏普
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')   # 年回报
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')   #  最大回撤分析器
cerebro.addobserver(bt.observers.Broker)  # 资金曲线
cerebro.addobserver(bt.observers.Trades)  #  交易胜率分布
cerebro.addobserver(bt.observers.DrawDown)  # 最大回撤

if not os.path.exists('回测缓存文件.pkl'):
    results = cerebro.run(save_my_data=True)  # 保存缓存,再运行
else :
    results = cerebro.run(load_my_data=True)  # 加载缓存,再运行

trade_analyzer = results[0].analyzers.trade_analyzer.get_analysis()
sharpe_ratio = results[0].analyzers.sharpe.get_analysis()
returns = results[0].analyzers.returns.get_analysis()
drawdown = results[0].analyzers.drawdown.get_analysis()
max_drawdown = drawdown.get('max', {}).get('drawdown', None)
print('=' * 88)
print( 交易分析(trade_analyzer) )
print("夏普比率:", sharpe_ratio)
print(f"最大回撤: {max_drawdown:.2f}%" if max_drawdown is not None else "没有回撤数据")
print(f"总回报率: {returns['rtot'] * 100:.2f}%")  # `rtot` 表示总回报率
print(f"年度回报率: {returns['rnorm'] * 100:.2f}%")  # `rnorm` 表示年度化回报率
print('=' * 88)
print("运行时间：", 小数2点(time.time() - 开始时间), "秒")
cerebro.plot(   )

每次预加载都需要消耗时间 :3.5589895248413086
2024-01-02
钱+股票： 500000.0
钱： 500000.0
我自己计算的钱: 500000
---------------------------------
··· 开盘前持仓展示：
持仓总成本: 0
持仓总价值: 0
---------------------------------
··· 先卖：
---------------------------------
··· 后买：
买入 601288 5.83 5.82 5.87 5.85 347908816.0
买入 000725 5.29 5.23 5.31 5.27 310637188.0
买入 600157 12.74 12.74 12.89 12.82 296362600.0
---------------------------------
··· 交易商处理：
--- order部分 --- 
日期: 2024-01-02  买入: 601288 订单 656  数量: 1000  买入价格: 5.86  手续费: 11.72  现金变动: -5871.72  现金余额: 494128.28
--- order部分 --- 
日期: 2024-01-02  买入: 000725 订单 657  数量: 1000  买入价格: 5.28  手续费: 10.56  现金变动: -5290.56  现金余额: 488837.72
--- order部分 --- 
日期: 2024-01-02  买入: 600157 订单 658  数量: 1000  买入价格: 12.83  手续费: 25.66  现金变动: -12855.66  现金余额: 475982.06
--- 触发 开仓 ---
买入:  日期 2024-01-02 测试，开仓时间 2024-01-03 00:00:00 订单 328 股票 601288 开仓价格: 5.86 开仓数量: 1000 持股天数: 0 毛利润: 0.0 净利润: -11.72
--- 触发 开仓 ---
买入:  日期 2024-01-02 测试，开仓时间 2024-01-03 00:00:00 订单 329 股票 000725 开仓价格: 5.28 开仓数量: 1000 持股天数

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 3 Axes>]]